## Data Handling

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
np.random.seed(4444)

In [2]:
df = pd.read_csv('../../../../../../jaeyeun/01_nh_poc/16_re_label/split_70_15_15_re_2.csv')

df.head()

,file_name,label_1,raw_text,dataset_n,prep_v9_text,label_2,label_3,train_val_test
0,R2003564.txt,0,보도일시 2020. 3. 13.(금) 조간(온라인 3. 12. 15:00)부터 보도...,0,온라인N 연구N 예산N 정부N 연구N 개발N r&dN 투자N 방향N 기준N 제시N ...,0,0,0
1,R1508023.txt,0,보\n도\n자\n료\n보도일시\n2015년 8월 3일 조간(8. 2. 12:00 이...,0,이후N 환경N 산업N 기술원N 기술N 평가N 세교N 전임N 연구원N 중소기업N 환경...,0,0,0
2,R1907682.txt,0,"보 도 자 료\n배포일시 2019. 7. 17.(수) 총 13매(본문 5, 참고 8...",0,참고N 도시N 교통N 신교통N 개발N 과장N 오송N 가능N 택시N 플랫폼N 혁신N ...,0,0,0
3,R2003319.txt,0,보도일시 2020. 3. 9.(월) 조간(온라인 3. 8. 12:00)부터 보도하여...,0,온라인N 거대N 공공N 연구N 정책N 우주N 기술N 서기관N 정부N 우주N 개발N ...,0,0,0
4,R2002452.txt,0,보 도 자 료\nhttp://www.motie.go.kr 2020년 2월 15일(토...,0,이후N 가능N 중견N 기업N 혁신N 과장N 우수N 중견N 강소N 기업N 지원N 기업...,0,0,0


* only for kdi dataset(exp1) & label_3

In [5]:
df = df[df['dataset_n'] == 0]

In [8]:
X_train_df = df[df['train_val_test'] == 0]
X_val_df = df[df['train_val_test'] == 1]
X_test_df = df[df['train_val_test'] == 2]
y_train = df[df['train_val_test'] == 0]['label_3']
y_val = df[df['train_val_test'] == 1]['label_3']
y_test = df[df['train_val_test'] == 2]['label_3']

In [9]:
print(X_train_df.shape)
print(X_val_df.shape)
print(X_test_df.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(7555, 8)
(1621, 8)
(1624, 8)
(7555,)
(1621,)
(1624,)


In [10]:
n_features = 100000
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.9, # 0.9 만큼의 문서 이상에서 나오면 거른다.
    min_df=5, # 5개 미만의 문서에서 나오면 거른다.
    sublinear_tf = True, # tf value를 완만하게 처리 (outlier 처리 효과)
    ngram_range = (1, 3),
    max_features=n_features)

* data is 'prep_v9_text'

In [11]:
X_train_tfidf = tfidf_vectorizer.fit(X_train_df['prep_v9_text'])
# X_test_tfidf = tfidf_vectorizer.fit_transform(prep_text_test)
# X_test_hash = hash_vectorizer.fit_transform(prep_text_test)

In [12]:
X_train = X_train_tfidf.transform(X_train_df['prep_v9_text'])
X_val = X_train_tfidf.transform(X_val_df['prep_v9_text'])
X_test = X_train_tfidf.transform(X_test_df['prep_v9_text'])

In [13]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(7555, 100000)
(1621, 100000)
(1624, 100000)


## Classify

* LightGBM RandomSearch

In [14]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgbm

In [15]:
lgbm_clf = lgbm.LGBMClassifier()

lgbm_param_grid = {'learning_rate': [.01, .015, .025, .05, .1],
#                   'Gamma': [.05, .1, .3, .5, .7, .9, 1],
                  'max_depth': [3, 5, 7, 9, 12, 15, 17, 25],
                  'min_child_weight': [1, 3, 5, 7],
                  'subsample': np.linspace(0.6, 1, 5)}

fit_params = {"early_stopping_rounds" : 20,
             "eval_metric" : "multi_error",
             "eval_set" : [(X_train, y_train), (X_val, y_val)]}

# Create a random search object
lgbm_random = RandomizedSearchCV(estimator = lgbm_clf,
                                param_distributions = lgbm_param_grid,
                                n_iter = 20, # n_iters in param combinations
                                scoring='accuracy',
                                n_jobs=-1,
                                cv = 5,
                                refit=True,
                                return_train_score = True,
                                verbose=10)

# Fit to the training data
lgbm_random.fit(X_train, y_train, **fit_params)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 16.5min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 21.5min
[Parallel(n_jobs=-1)]: Done  48 out of 100 | elapsed: 28.5min remaining: 30.8min
[Parallel(n_jobs=-1)]: Done  59 out of 100 | elapsed: 32.8min remaining: 22.8min
[Parallel(n_jobs=-1)]: Done  70 out of 100 | elapsed: 37.2min remaining: 15.9min
[Parallel(n_jobs=-1)]: Done  81 out of 100 | elapsed: 39.9min remaining:  9.3min
[Parallel(n_jobs=-1)]: Done  92 out of 100 | elapsed: 43.7min remaining:  3.8min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 48.3min finished


[1]	training's multi_error: 0.342158	training's multi_logloss: 1.92824	valid_1's multi_error: 0.37446	valid_1's multi_logloss: 1.94868
Training until validation scores don't improve for 20 rounds
[2]	training's multi_error: 0.173527	training's multi_logloss: 1.70064	valid_1's multi_error: 0.229488	valid_1's multi_logloss: 1.74134
[3]	training's multi_error: 0.127333	training's multi_logloss: 1.52344	valid_1's multi_error: 0.193708	valid_1's multi_logloss: 1.5816
[4]	training's multi_error: 0.104169	training's multi_logloss: 1.38011	valid_1's multi_error: 0.178285	valid_1's multi_logloss: 1.45531
[5]	training's multi_error: 0.094904	training's multi_logloss: 1.25925	valid_1's multi_error: 0.173967	valid_1's multi_logloss: 1.34697
[6]	training's multi_error: 0.0863005	training's multi_logloss: 1.15504	valid_1's multi_error: 0.167181	valid_1's multi_logloss: 1.25401
[7]	training's multi_error: 0.0815354	training's multi_logloss: 1.06447	valid_1's multi_error: 0.164713	valid_1's multi_logl

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.015, 0.025,
                                                          0.05, 0.1],
                                        'max_depth': [3, 5, 7, 9, 12, 15, 17,
                                                      25],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'subsample': array([0.6, 0.7, 0.8, 0.9, 1. ])},
                   return_train_score=True, scoring='accuracy', verbose=10)

In [16]:
pred_train = lgbm_random.best_estimator_.predict(X_train)
pred_val = lgbm_random.best_estimator_.predict(X_val)
pred_test = lgbm_random.best_estimator_.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))
print(accuracy_score(y_test, pred_test))

0.999602911978822
0.8747686613201727
0.8842364532019704


In [18]:
cv_result_df = pd.DataFrame(lgbm_random.cv_results_)

df_list = []
for i in range(20):
    df_list.append(pd.DataFrame([cv_result_df.loc[i, "params"]]))
    
param_table = pd.concat(df_list)

param_table['mean_test_score'] = cv_result_df['mean_test_score'].values

param_table.sort_values(by='mean_test_score', axis=0)

,subsample,min_child_weight,max_depth,learning_rate,mean_test_score
0,0.7,1,7,0.010,0.829517
0,0.6,7,25,0.010,0.830576
0,0.9,1,25,0.010,0.835606
0,0.9,3,12,0.010,0.836664
0,0.7,3,12,0.010,0.836664
0,1.0,7,12,0.015,0.841032
0,0.8,5,9,0.015,0.841165
0,0.7,3,7,0.015,0.841694
0,1.0,7,12,0.025,0.842356
0,0.6,7,15,0.025,0.843018
